## Calculate the Final density with TL & MF approximation

In [1]:
from sklearn.model_selection import ParameterGrid
import numpy as np
from multiprocessing import Pool

from scipy.optimize import root
from tqdm import tqdm

import math
import time

import warnings
warnings.filterwarnings("ignore")

import scipy.stats as scis
ppmf  = scis.poisson.pmf

import matplotlib.pyplot as plt
import pickle

import os

from custom_functions import bipmf, MF_2m, TL_2m, TL, MF

### fix the modularity and get the phase diagram between $\omega_1$ & $\omega_2$ under TL approximation

In [ ]:
tb = time.time()

w1_num = 30
w2_num = 30

#     mu_num = 40
#     mu_list = np.linspace(0, 0.5, num_num)

mu_list = list([0.75, 0.85, 0.95, 0])
theta_ = 0.1
w1_list = np.linspace(0, 1, w1_num).tolist()
w2_list = np.linspace(0, 1, w2_num).tolist()  
lam_ = 0

for mu_ in mu_list:

    print(f"multiprocess begins, mu = {mu_}")

    param_grid = {"mu" : list([mu_]), "w1" : w1_list, "w2" : w2_list, \
                  "lam" : list([lam_]), "theta" : list([theta_])}
    grid = ParameterGrid( param_grid)
    p = Pool()
    jobs = [p.map_async(TL, (grid[i], )) for i in range( len(grid))]
    results = []
    for job in tqdm(jobs):
        results.append( job.get())

    file_path = os.path.join( os.getcwd(), f"RHO_N_mu{mu_}grid{w1_num}.dat")            
    pickle.dump(results,open( file_path, "wb"))
    tm = time.time()

    print( (tm - tb)/(60*60*24), "day")
    p.close()
    p.join()

  0%|                                                                                          | 0/900 [00:00<?, ?it/s]

multiprocess begins, mu = 0.75


  1%|▍                                                                               | 5/900 [01:15<4:55:25, 19.80s/it]

### fix the $\omega_1$ and get the phase diagram between $\mu$ & $\omega_2$ under TL approximation

### fix the $\omega_2$ and get the phase diagram between $\mu$ & $\omega_1$ under TL approximation

### calculate the final density **VS** $\mu$ under MF approximation

In [ ]:
%%time
# MF approximation------------------------------------------------------------

w1_num = 100
w2_num = 100  
mu_num = 100
mu_list = np.linspace(0, 1, mu_num)

#     mu_list = list([0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5])
theta_ = 0.25
w1_list = np.linspace(0, 1, w1_num).tolist()
w2_list = np.linspace(0, 1, w2_num).tolist() 

w1_ = 0.8
w2_ = 1
lam_ = 0

param_grid = {"mu" : mu_list, "w2" : list([w2_]), "w1" : list([w1_]), \
              "lam" : list([lam_]), "theta" : list([theta_])}
grid = ParameterGrid( param_grid)

p = Pool()    
jobs = [p.map_async(MF, (grid[i], )) for i in range(len(grid))]
results = []
for job in tqdm(jobs):
    results.append( job.get())

file_path = os.path.join( os.getcwd(), "data",f"MFw1{w1_}w2{w2_}theta{theta_}.dat")
pickle.dump(results, open(file_path, "wb"))

p.close()
p.join()
        
# MF approximation------------------------------------------------------------ 

## save data as .h5 data

In [ ]:
def reSaveData(mu_, N = 40):
    
    w1_list = np.linspace(0, 1, N).tolist()
    w2_list = np.linspace(0, 1, N).tolist()
    
    rho_0 = 0.17
    theta = 0.1
#     file_path = os.path.join(os.getcwd(), "data", f"RHO_N_mu{mu_}lam0.dat")
    
    file_path = os.path.join(os.getcwd(), "data", f"RHO_N_mu{mu_}grid30.dat")
    
    result_new = pickle.load(open(file_path, "rb"))
    w1_mesh, w2_mesh = np.meshgrid(w1_list, w2_list)
    rho = np.zeros((N, N, 2))
    
    for i in range(N):
        for j in range(N):            
            rho[j, i, :] = result_new[i + j * N][0][0]
                       
    rho_Inf =  rho[:, :, :].mean(axis = 2).T
    
    file_pathNew = os.path.join(os.getcwd(), "dataNewStruc", f"mu{mu_}.h5")
    
    f = h5py.File(file_pathNew, 'w')
    
    f.create_dataset('w1', data = w1_list)
    f.create_dataset('w2', data = w2_list)
    f.create_dataset('rho', data = rho_Inf)
    f.create_dataset('rho_0', data = rho_0)
    f.create_dataset('theta', data = theta)
    f.create_dataset('Method', data = "TL approximation")        
    f.close()

for mu_ in list([0.95]):
    reSaveData(mu_, 30)

# fileLoc = r"dataNewStruc/mu0.2.h5"

# f = h5py.File(fileLoc, 'r')
# w1_ = f['w1'][()]
# w2_ = f['w2'][()]
# rho = f["rho"][()]

# ll = f["Method"][()]
    
# f.close()

# fileLoc =  r"dataNewStruc/mu0.2.h5"

# w1_ = f["w1"][()]
# w2_ = f['w2'][()]
# rho = f["rho"][()]
# rho_0 = f["rho_0"][()]
# theta = f["theta"][()]
# rho = f["rho"][()]